# TensorFlow to PyTorch

In [1]:
import pickle
import random
import warnings
import tensorflow as tf
import torch
import torch.nn as nn
import torchvision.models as models

In [2]:
random.seed(123)
warnings.filterwarnings('ignore')

In [3]:
tensorFlow_model = tf.keras.models.load_model('vgg_16_model.h5')

In [4]:
# extract the weights
weights = {}
for layer in tensorFlow_model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
        weights[layer.name] = {
            "kernel": layer.get_weights()[0],  
            "bias": layer.get_weights()[1]     
        }

## Create the PyTorch Version

In [5]:
# define the pytorch model version
vgg16_pytorch = models.vgg16(pretrained=False)

# move weights to the PyTorch
with torch.no_grad():
    for (name, param), (layer_name, weight) in zip(vgg16_pytorch.named_parameters(), weights.items()):
        if 'conv' in name:
            # transpose the weights 
            param.data = torch.Tensor(weight['kernel']).permute(3, 2, 0, 1) 
        elif 'fc' in name:
            param.data = torch.Tensor(weight['kernel']).T  
            vgg16_pytorch.classifier[6].bias.data = torch.Tensor(weight['bias']) 

## Save the Model

In [6]:
# define the filename to save the vgg-16 model
fileName = "vgg_16_model_pytorch.pkl"
# save the model
with open(fileName, 'wb') as file:
    pickle.dump(vgg16_pytorch ,file)